# **Разметка текстов**
### **1.** Использование одной из библиотек (nltk , mystem , pymorphy) для разметки частей речи и измерения качества для нескольких текстов из размеченных корпусов для русского языка.

### **2.** Разметка частей речи и именованных сущностей, анализ ошибок алгоритма для нескольких текстов новостного датасета.

In [ ]:
import os
import spacy
from spacy import displacy
import pandas as pd
import opencorpora
import numpy as np
import lxml
from lxml import objectify, etree, html
from xml.etree import ElementTree as xml
import urllib3
from io import StringIO, BytesIO
import re
import codecs
from pymystem3 import Mystem
import copy
import nltk
import string
import sklearn
import scipy
import itertools
import json
import random
import warnings
warnings.filterwarnings("ignore")

# Часть 1
**Выбрать несколько текстов из размеченных корпусов для русского языка. Использовать одну
из библиотек ( nltk , mystem , pymorphy ) для разметки частей речи, измерить качество.**

В качестве данных был взят корпус **OpenCorpora.org**

## Чтение и подготовка данных

In [ ]:
corpus = opencorpora.load('annot.opcorpora.xml')

Сначала я учусь вытаскивать разную информацию из корпуса.

In [ ]:
len(corpus.docs)

4030

In [ ]:
len(corpus.sentences)

110306

В данном размеченном корпусе содержится 110 306 предложений.

In [ ]:
corpus.sentences[17]

<Sentence id=18 source='Это произошло году в 2005-м, когда на смену героям, кочующим из одного ток-шоу в другое и по дороге заглядывающим в «Школу злословия», к Татьяне Никитичне и Авдотье Андреевне стал приходить совсем другой народ: политологи, публицисты, причем практически исключительно либеральных убеждений.'>

In [ ]:
doc = corpus[42]
doc

<Doc id=44 tokens:2502 name='18801 Хитрость духа'>

In [ ]:
doc.source

'Хитрость духа  Почему князь Владимир крестил Русь\n\n28 июля православная церковь чтит память равноапостольного князя Владимира, в крещении Василия.  В этом году впервые на государственном уровне празднуют этот день как День Крещения Руси.  Крестившись сам, Владимир затем крестил своих поданных, отсюда пошла русская православная цивилизация.  В том числе и ее проблемы, возможно, связанные с некоторыми свойствами характера Владимира Святославича, прозванного в былинах Красно Солнышко.\n\nУ князя Святослава Игоревича было три сына: Ярополк, Олег и Владимир.  Имена матерей первых двух неизвестны, а Владимира родила Малуша, которая была рабыней, ключницей матери Святослава Ольги (см. здесь ).  В «Повести временных лет» сказано, что Малуша «была сестра Добрыни, а отец им был Малк Любечанин».  Кто такой — неизвестно, возможно, тот самый древлянин Мал, которой убил князя Игоря и которому Ольга мстила.  Если так, то Малуша была пленной древлянкой.  Но как бы там ни было, девушка была хороша с

In [ ]:
len(doc.sentences)

160

In [ ]:
doc.tokens

[<Token id=64838 source='Хитрость'>,
 <Token id=64839 source='духа'>,
 <Token id=64840 source='Почему'>,
 <Token id=64841 source='князь'>,
 <Token id=64842 source='Владимир'>,
 <Token id=64843 source='крестил'>,
 <Token id=64844 source='Русь'>,
 <Token id=64845 source='28'>,
 <Token id=64846 source='июля'>,
 <Token id=64847 source='православная'>,
 <Token id=64848 source='церковь'>,
 <Token id=64849 source='чтит'>,
 <Token id=64850 source='память'>,
 <Token id=64851 source='равноапостольного'>,
 <Token id=64852 source='князя'>,
 <Token id=64853 source='Владимира'>,
 <Token id=64854 source=','>,
 <Token id=64855 source='в'>,
 <Token id=64856 source='крещении'>,
 <Token id=64857 source='Василия'>,
 <Token id=64858 source='.'>,
 <Token id=64859 source='В'>,
 <Token id=64860 source='этом'>,
 <Token id=64861 source='году'>,
 <Token id=64862 source='впервые'>,
 <Token id=64863 source='на'>,
 <Token id=64864 source='государственном'>,
 <Token id=64865 source='уровне'>,
 <Token id=64866 source

In [ ]:
sent = doc.sentences[6]
sent

<Sentence id=3439 source='У князя Святослава Игоревича было три сына: Ярополк, Олег и Владимир.'>

In [ ]:
sent.tokens

[<Token id=64912 source='У'>,
 <Token id=64913 source='князя'>,
 <Token id=64914 source='Святослава'>,
 <Token id=64915 source='Игоревича'>,
 <Token id=64916 source='было'>,
 <Token id=64917 source='три'>,
 <Token id=64918 source='сына'>,
 <Token id=64919 source=':'>,
 <Token id=64920 source='Ярополк'>,
 <Token id=64921 source=','>,
 <Token id=64922 source='Олег'>,
 <Token id=64923 source='и'>,
 <Token id=64924 source='Владимир'>,
 <Token id=64925 source='.'>]

In [ ]:
token = sent[1]
token.source

'князя'

In [ ]:
token.grammemes[0]

'NOUN'

Для дальнейшей работы я беру **10 000 предложений** из своего корпуса.

In [ ]:
document = corpus.sentences[0:10000]
document[0]

<Sentence id=1 source='«Школа злословия» учит прикусить язык'>

Привожу данные к формату, который нужно подать в языковые модели. 
Предложения разбиваются на токены, и каждому токену в соответствие ставится POS-тег.

In [ ]:
data = list()

for sent in document:
    sentence = list()
    for token in sent.tokens:
        t = (token.source, token.grammemes[0])
        sentence.append(t)
    data.append(sentence)

In [ ]:
len(data)

10000

Пример данных.

In [ ]:
data[0:3]

[[('«', 'PNCT'),
  ('Школа', 'NOUN'),
  ('злословия', 'NOUN'),
  ('»', 'PNCT'),
  ('учит', 'VERB'),
  ('прикусить', 'INFN'),
  ('язык', 'NOUN')],
 [('Сохранится', 'VERB'),
  ('ли', 'PRCL'),
  ('градус', 'NOUN'),
  ('дискуссии', 'NOUN'),
  ('в', 'PREP'),
  ('новом', 'ADJF'),
  ('сезоне', 'NOUN'),
  ('?', 'PNCT')],
 [('Великолепная', 'ADJF'),
  ('«', 'PNCT'),
  ('Школа', 'NOUN'),
  ('злословия', 'NOUN'),
  ('»', 'PNCT'),
  ('вернулась', 'VERB'),
  ('в', 'PREP'),
  ('эфир', 'NOUN'),
  ('после', 'PREP'),
  ('летних', 'ADJF'),
  ('каникул', 'NOUN'),
  ('в', 'PREP'),
  ('новом', 'ADJF'),
  ('формате', 'NOUN'),
  ('.', 'PNCT')]]

Делю данные на train, test. 80% данных - обучающая выборка, 20% - тестовая.

In [ ]:
train_data = data[:8000]
test_data = data[8000:]

len(train_data), len(test_data)

(8000, 2000)

## Обучение языковой модели

Сделаю комбинацию из нескольких анализаторов. В начале использую анализатор по умолчанию, он автоматически ставит 1 тег каждому слову. В качестве тега я взяла NOUN, т.к. имя существительное - это самая часто встречаемая часть речи в предложениях. Далее, последовательно применяю Unigram, Bigram и Trigram теггеры.

In [ ]:
default_tagger = nltk.DefaultTagger('NOUN')
unigram_tagger = nltk.UnigramTagger(train_data, backoff=default_tagger)
bigram_tagger = nltk.BigramTagger(train_data, backoff=unigram_tagger)
trigram_tagger = nltk.TrigramTagger(train_data, backoff=bigram_tagger)

## Оценка точности

Оценю 2 модели.

In [ ]:
bigram_tagger.evaluate(test_data)

0.8705773066309663

In [ ]:
trigram_tagger.evaluate(test_data)

0.8698942651840114

Точность у Bigram model оказалась выше, чем у Trigram model.

Пробую предсказать теги для неразмеченных предложений, не участвовавших в обучении и оценке.

In [ ]:
doc_ = corpus.sentences[10100:10200]

test_notags, test_tags = list(), list()
for s in doc_:
    sent, tags = list(), list()
    for token in s.tokens:
        sent.append(token.source)
        tags.append(token.grammemes[0])
    test_notags.append(sent)
    test_tags.append(tags)

len(test_notags), len(test_tags)

(100, 100)

Проверю 2 модели на 2 предложениях.

* **1-ое предложение**

In [ ]:
print(bigram_tagger.tag(test_notags[99]))

[('Что', 'CONJ'), ('случилось', 'VERB'), ('с', 'PREP'), ('поездом', 'NOUN'), ('?', 'PNCT')]


In [ ]:
print(trigram_tagger.tag(test_notags[99]))

[('Что', 'CONJ'), ('случилось', 'VERB'), ('с', 'PREP'), ('поездом', 'NOUN'), ('?', 'PNCT')]


Сверю с настоящей разметкой.

In [ ]:
test_tags[99]

['CONJ', 'VERB', 'PREP', 'NOUN', 'PNCT']

Всё совпало.

* **2-ое предложение**

In [ ]:
print(bigram_tagger.tag(test_notags[4]))
print(trigram_tagger.tag(test_notags[4]))
print(test_tags[4])

[('25', 'NUMB'), ('марта', 'NOUN'), ('2010', 'NUMB'), ('года', 'NOUN'), ('Рустам', 'NOUN'), ('Нургалиевич', 'NOUN'), ('Минниханов', 'NOUN'), ('принес', 'VERB'), ('присягу', 'NOUN'), ('и', 'CONJ'), ('вступил', 'VERB'), ('в', 'PREP'), ('должность', 'NOUN'), ('президента', 'NOUN'), ('Руспублики', 'NOUN'), ('Татарстан', 'NOUN'), ('.', 'PNCT')]
[('25', 'NUMB'), ('марта', 'NOUN'), ('2010', 'NUMB'), ('года', 'NOUN'), ('Рустам', 'NOUN'), ('Нургалиевич', 'NOUN'), ('Минниханов', 'NOUN'), ('принес', 'VERB'), ('присягу', 'NOUN'), ('и', 'CONJ'), ('вступил', 'VERB'), ('в', 'PREP'), ('должность', 'NOUN'), ('президента', 'NOUN'), ('Руспублики', 'NOUN'), ('Татарстан', 'NOUN'), ('.', 'PNCT')]
['NUMB', 'NOUN', 'NUMB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'CONJ', 'VERB', 'PREP', 'NOUN', 'NOUN', 'UNKN', 'NOUN', 'PNCT']


Теги определены правильно в каждом случае.

Сохраню лучшую модель.

In [ ]:
from pickle import dump
output = open('bigram.pkl', 'wb')
dump(bigram_tagger, output, -1)
output.close()

# Часть 2
**Для нескольких текстов из собранного новостного датасета выполнить разметку частей речи и
именованных сущностей, проанализировать ошибки алгоритма.**

## Чтение и предобработка данных

In [ ]:
xmlstr = codecs.open('dataPatina.xml', encoding='utf-8', mode='r').read()
root = etree.fromstring(xmlstr)

catalog = list()
for element_lvl1 in root:
    article = {}
    for element_lvl2 in element_lvl1:
        txt = element_lvl2.text
        article[element_lvl2.tag] = '' if txt is None else txt
    catalog.append(article)

In [ ]:
def preprocessing(text):
    return re.sub( '\s+', ' ', text).strip()

data = list()
for i in range(len(catalog)):
    data.append(preprocessing(catalog[i]['text']))

In [ ]:
len(data)

200

Я хочу найти короткие тексты, чтобы было удобнее анализировать разметку.

In [ ]:
ind = list()
for i in range(len(data)):
    if len(data[i])<900 and len(data[i])>100:
        ind.append(i)

len(ind)

71

Из 71 текста выберу 2 для анализа.

In [ ]:
data[ind[6]]

"Video The nominations for this year's Academy Awards have been announced, Joker leading the pack with 11 nods. The comic book villain origin story is up for best picture, best director and best actor for Joaquin Phoenix, plus eight other awards. The Irishman, 1917 and Once Upon a Time in Hollywood follow with 10 nominations each. But who else has been nominated?"

In [ ]:
print(data[1]) # the same - data[ind[0]]

This story is nominated for a Webby Award for Best Film & Video. Vote here. BBC Earth is also nominated for a Webby, for Best Science Website. Vote here. In theory this is not the only Universe that might exist, and in many others, identical copies of us can be found. The question is, how do we get there? BBC Earth's Melissa Hogenboom goes on the hunt for her cosmic twin. Melissa Hogenboom is BBC Earth's feature writer. She is @melissasuzanneh on Twitter. Video produced by Pierangelo Pirak; he is @ppirak on twitter. Join over five million BBC Earth fans by liking us on Facebook, or follow us on Twitter and Instagram. If you liked this story, sign up for the weekly bbc.com features newsletter called "If You Only Read 6 Things This Week". A handpicked selection of stories from BBC Future, Earth, Culture, Capital, Travel and Autos, delivered to your inbox every Friday.


## Разметка

Для каждого текста 
* построю таблицу, в которой для каждого токена текста показывается его лемма, pos-tag, детализированный pos-tag, синтаксическая зависимость, форма слова и принадлежность токена к набору стоп-слов; 
* сделаю dependency parsing;
* named entity recognition.

In [ ]:
nlp = spacy.load('en_core_web_sm')
PIPELINE = ['tagger', 'parser', 'ner']
names_col = ['TEXT', 'LEMMA', 'POS', 'TAG', 'DEP', 'SHAPE', 'STOP']

* **ТЕКСТ 1**

In [ ]:
doc1 = nlp(data[1])

In [ ]:
doc1

This story is nominated for a Webby Award for Best Film & Video. Vote here. BBC Earth is also nominated for a Webby, for Best Science Website. Vote here. In theory this is not the only Universe that might exist, and in many others, identical copies of us can be found. The question is, how do we get there? BBC Earth's Melissa Hogenboom goes on the hunt for her cosmic twin. Melissa Hogenboom is BBC Earth's feature writer. She is @melissasuzanneh on Twitter. Video produced by Pierangelo Pirak; he is @ppirak on twitter. Join over five million BBC Earth fans by liking us on Facebook, or follow us on Twitter and Instagram. If you liked this story, sign up for the weekly bbc.com features newsletter called "If You Only Read 6 Things This Week". A handpicked selection of stories from BBC Future, Earth, Culture, Capital, Travel and Autos, delivered to your inbox every Friday.

In [ ]:
df1 = pd.DataFrame(columns=names_col)
for token in doc1:
    df1 = df1.append({'TEXT': token.text, 'LEMMA': token.lemma_, 'POS': token.pos_, 'TAG': token.tag_, 
                            'DEP': token.dep_, 'SHAPE': token.shape_, 'STOP': token.is_stop}, ignore_index=True)

In [ ]:
df1.head(100)

,TEXT,LEMMA,POS,TAG,DEP,SHAPE,STOP
0,This,this,DET,DT,det,Xxxx,True
1,story,story,NOUN,NN,nsubjpass,xxxx,False
2,is,be,AUX,VBZ,auxpass,xx,True
3,nominated,nominate,VERB,VBN,ROOT,xxxx,False
4,for,for,ADP,IN,prep,xxx,True
5,a,a,DET,DT,det,x,True
6,Webby,Webby,PROPN,NNP,compound,Xxxxx,False
7,Award,Award,PROPN,NNP,pobj,Xxxxx,False
8,for,for,ADP,IN,prep,xxx,True
9,Best,Best,PROPN,NNP,compound,Xxxx,False


В основном POS-теги расставлены правильно. Однако, ник акаунта в твиттере @melissasuzanneh, я бы определила к NOUN или X(other), а не к adjective. Также, если Science Website отнесены к имени собственному, то и Best тоже должно быть именем собственным. 

In [ ]:
for sent in doc1.sents:
    print(sent)
    displacy.render(sent, style='dep', jupyter=True, options={'compact': True, "bg": "ivory"})

This story is nominated for a Webby Award for Best Film & Video.


Vote here.


BBC Earth is also nominated for a Webby, for Best Science Website.


Vote here.


In theory this is not the only Universe that might exist, and in many others, identical copies of us can be found.


...

Join over five million BBC Earth fans by liking us on Facebook, or follow us on Twitter and Instagram.


If you liked this story, sign up for the weekly bbc.com


features newsletter called "


If You Only Read 6 Things This Week".


A handpicked selection of stories from BBC Future, Earth, Culture, Capital, Travel and Autos, delivered to your inbox every Friday.


На мой взгляд зависимости определяются правильно.

In [ ]:
displacy.render(doc1, style='ent', jupyter=True)

С поиском именованных сущностей дела обстоят хуже, довольно много ошибок. Webby Award должно быть - EVENT или WORK_OF_ART, причём a,for не должны быть выделены, а Twitter, Facebook должны определяться как тег - ORG. Best Film & Video, BBC Earth, Best Science Website, BBC Future, @melissasuzanneh я также отнесла бы к другим категориям. Не выделены такие сущности как: Instagram, @ppirak, five million, Universe. Travel не верно определилось, я бы вообще не отнесла это слово даже к сущности.  Если не учитывать контекст, то Webby можно принять за имя, иначе это должно относиться к другой категории.

* **ТЕКСТ 2**

In [ ]:
doc2 = nlp(data[ind[6]])

In [ ]:
doc2

Video The nominations for this year's Academy Awards have been announced, Joker leading the pack with 11 nods. The comic book villain origin story is up for best picture, best director and best actor for Joaquin Phoenix, plus eight other awards. The Irishman, 1917 and Once Upon a Time in Hollywood follow with 10 nominations each. But who else has been nominated?

In [ ]:
df2 = pd.DataFrame(columns=names_col)
for token in doc2:
    df2 = df2.append({'TEXT': token.text, 'LEMMA': token.lemma_, 'POS': token.pos_, 'TAG': token.tag_, 
                            'DEP': token.dep_, 'SHAPE': token.shape_, 'STOP': token.is_stop}, ignore_index=True)

In [ ]:
df2

,TEXT,LEMMA,POS,TAG,DEP,SHAPE,STOP
0,Video,Video,PROPN,NNP,ROOT,Xxxxx,False
1,The,the,DET,DT,det,Xxx,True
2,nominations,nomination,NOUN,NNS,nsubjpass,xxxx,False
3,for,for,ADP,IN,prep,xxx,True
4,this,this,DET,DT,det,xxxx,True
5,year,year,NOUN,NN,poss,xxxx,False
6,'s,'s,PART,POS,case,'x,True
7,Academy,Academy,PROPN,NNP,compound,Xxxxx,False
8,Awards,Awards,PROPN,NNPS,pobj,Xxxxx,False
9,have,have,AUX,VBP,aux,xxxx,True


Я думаю, что слово villain не относится к proper noun, а относится просто к noun.

In [ ]:
for sent in doc2.sents:
    print(sent)
    displacy.render(sent, style='dep', jupyter=True, options={'compact': True, "bg": "ivory"})

Video


The nominations for this year's Academy Awards have been announced, Joker leading the pack with 11 nods.


The comic book villain origin story is up for best picture, best director and best actor for Joaquin Phoenix, plus eight other awards.


The Irishman, 1917 and Once Upon a Time in Hollywood follow with 10 nominations each.


But who else has been nominated?


In [ ]:
displacy.render(doc2, style='ent', jupyter=True)

Joker должен быть определён как Person. Также в данном контексте, Irishman, 1917, Once Upon a Time in Hollywood, это названия фильмов, и они должны определяться как WORK_OF_ART. Если контекст не учитывать, то категории соотнесены верно.

**Вывод:** Алгоритм поиска именнованных сущностей нужно улучшать, или расширять его словарь. А определение POS-тегов и синтаксических зависимостей работает довольно хорошо.